In [3]:
import pandas as pd
import numpy as np

In [4]:
!pip install pyarrow


In [5]:

df_january = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_february = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [6]:
# Display the number of columns for each month
num_columns_january = len(df_january.columns)
num_columns_february = len(df_february.columns)
num_columns_january, num_columns_february

(19, 19)

In [7]:
# Convert pickup and dropoff times to datetime
df_january['tpep_pickup_datetime'] = pd.to_datetime(df_january['tpep_pickup_datetime'])
df_january['tpep_dropoff_datetime'] = pd.to_datetime(df_january['tpep_dropoff_datetime'])

# Compute the duration in minutes
df_january['duration'] = (df_january['tpep_dropoff_datetime'] - df_january['tpep_pickup_datetime']).dt.total_seconds() / 60

# Calculate the standard deviation of the trip durations
std_duration_january = df_january['duration'].std()
std_duration_january

42.594351241920904

In [8]:
# Filter records with duration between 1 and 60 minutes (inclusive)
filtered_df_january = df_january[(df_january['duration'] >= 1) & (df_january['duration'] <= 60)]

# Calculate the fraction of records left
fraction_left = len(filtered_df_january) / len(df_january)
fraction_left

0.9812202822125979

In [9]:
# Convert location IDs to strings
filtered_df_january['PULocationID'] = filtered_df_january['PULocationID'].astype(str)
filtered_df_january['DOLocationID'] = filtered_df_january['DOLocationID'].astype(str)

/tmp/ipykernel_75213/2103125816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_january['PULocationID'] = filtered_df_january['PULocationID'].astype(str)
/tmp/ipykernel_75213/2103125816.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_january['DOLocationID'] = filtered_df_january['DOLocationID'].astype(str)


In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [12]:
# Select the relevant columns
X_train = filtered_df_january[['PULocationID', 'DOLocationID']]
y_train = filtered_df_january['duration']


In [19]:
# Fit a OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = ohe.fit_transform(X_train)
# Get the dimensionality of the feature matrix
dimensionality = X_train_encoded.shape[1]
dimensionality

515

In [20]:
# Prepare the target variable
y = filtered_df_january['duration'].values

# Train a linear regression model
model = LinearRegression()
model.fit(X_train_encoded, y)

# Predict on the training data
y_pred = model.predict(X_train_encoded)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
rmse

7.649261027844195

In [15]:
# Convert pickup and dropoff times to datetime
df_february['tpep_pickup_datetime'] = pd.to_datetime(df_february['tpep_pickup_datetime'])
df_february['tpep_dropoff_datetime'] = pd.to_datetime(df_february['tpep_dropoff_datetime'])

# Compute the duration in minutes
df_february['duration'] = (df_february['tpep_dropoff_datetime'] - df_february['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter records with duration between 1 and 60 minutes (inclusive)
filtered_df_february = df_february[(df_february['duration'] >= 1) & (df_february['duration'] <= 60)]

# Convert location IDs to strings
filtered_df_february['PULocationID'] = filtered_df_february['PULocationID'].astype(str)
filtered_df_february['DOLocationID'] = filtered_df_february['DOLocationID'].astype(str)


/tmp/ipykernel_75213/3027571160.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_february['PULocationID'] = filtered_df_february['PULocationID'].astype(str)
/tmp/ipykernel_75213/3027571160.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_february['DOLocationID'] = filtered_df_february['DOLocationID'].astype(str)


In [16]:
# Select the relevant columns
X_val = filtered_df_february[['PULocationID', 'DOLocationID']]
y_val = filtered_df_february['duration']

In [21]:
# Transform the validation data using the fitted OneHotEncoder

X_val_encoded = ohe.transform(X_val)


In [22]:

# Predict on the validation data
y_pred_february = model.predict(X_val_encoded)

# Calculate the RMSE for the validation data
rmse_february = np.sqrt(mean_squared_error(y_val, y_pred_february))
rmse_february

7.811832672340139